# NUMCHAIN v1.4.2 — Closed-Alpha Notebook Export
This notebook scaffolds a runnable repo folder (`./numchain/`) with:
- `docker-compose.yml` (5-node cluster)
- `Makefile`
- `README.md`
- helper scripts under `tools/`

**Important:** This notebook does **not** reconstruct your `numchain_v142.py` if you do not provide it.
If you already have your `numchain_v142.py` (the v1.4.2 single-file kernel), place it in the same directory as this notebook, or paste it into the designated cell below.


In [ ]:

import pathlib

ROOT = pathlib.Path("numchain").resolve()
TOOLS = ROOT / "tools"
LOGS = ROOT / "logs"

for p in [ROOT, TOOLS, LOGS]:
    p.mkdir(parents=True, exist_ok=True)

print("Repo folder:", ROOT)


## 1) Provide `numchain_v142.py`
If you already have `numchain_v142.py` in the current working directory, the next cell will copy it into `./numchain/`.
Otherwise, paste your v1.4.2 script into the string below (keep it exact), then run the cell.


In [ ]:

import pathlib, shutil

ROOT = pathlib.Path("numchain").resolve()
SRC = pathlib.Path("numchain_v142.py")
DST = ROOT / "numchain_v142.py"

if SRC.exists():
    shutil.copy2(SRC, DST)
    print("Copied existing numchain_v142.py ->", DST)
else:
    NUMCHAIN_V142 = r'''# PASTE YOUR v1.4.2 numchain_v142.py HERE (entire file).
# If you leave this placeholder, the notebook will not be runnable.
raise SystemExit("numchain_v142.py not provided. Paste the full v1.4.2 code into this cell or place numchain_v142.py beside the notebook.")
'''
    DST.write_text(NUMCHAIN_V142, encoding="utf-8")
    print("Wrote placeholder numchain_v142.py ->", DST)


## 2) Write `docker-compose.yml` (5-node localhost cluster)

In [ ]:

import pathlib

ROOT = pathlib.Path("numchain").resolve()

docker_compose = '''version: "3.9"

services:
  node0:
    image: python:3.12-slim
    working_dir: /app
    volumes:
      - ./:/app
      - ./logs/node0:/data
    command: >
      bash -lc "
        python numchain_v142.py deps &&
        python numchain_v142.py run
          --genesis /app/genesis.json
          --node-index 0
          --rpc-host 0.0.0.0 --rpc 8000
          --gossip-host 0.0.0.0 --gossip 9000
          --peers node1:9001,node2:9002,node3:9003,node4:9004
          --log /data/node0.log
      "
    ports:
      - "8000:8000"
    depends_on:
      - node1
      - node2
      - node3
      - node4

  node1:
    image: python:3.12-slim
    working_dir: /app
    volumes:
      - ./:/app
      - ./logs/node1:/data
    command: >
      bash -lc "
        python numchain_v142.py deps &&
        python numchain_v142.py run
          --genesis /app/genesis.json
          --node-index 1
          --rpc-host 0.0.0.0 --rpc 8001
          --gossip-host 0.0.0.0 --gossip 9001
          --peers node0:9000,node2:9002,node3:9003,node4:9004
          --log /data/node1.log
      "
    ports:
      - "8001:8001"

  node2:
    image: python:3.12-slim
    working_dir: /app
    volumes:
      - ./:/app
      - ./logs/node2:/data
    command: >
      bash -lc "
        python numchain_v142.py deps &&
        python numchain_v142.py run
          --genesis /app/genesis.json
          --node-index 2
          --rpc-host 0.0.0.0 --rpc 8002
          --gossip-host 0.0.0.0 --gossip 9002
          --peers node0:9000,node1:9001,node3:9003,node4:9004
          --log /data/node2.log
      "
    ports:
      - "8002:8002"

  node3:
    image: python:3.12-slim
    working_dir: /app
    volumes:
      - ./:/app
      - ./logs/node3:/data
    command: >
      bash -lc "
        python numchain_v142.py deps &&
        python numchain_v142.py run
          --genesis /app/genesis.json
          --node-index 3
          --rpc-host 0.0.0.0 --rpc 8003
          --gossip-host 0.0.0.0 --gossip 9003
          --peers node0:9000,node1:9001,node2:9002,node4:9004
          --log /data/node3.log
      "
    ports:
      - "8003:8003"

  node4:
    image: python:3.12-slim
    working_dir: /app
    volumes:
      - ./:/app
      - ./logs/node4:/data
    command: >
      bash -lc "
        python numchain_v142.py deps &&
        python numchain_v142.py run
          --genesis /app/genesis.json
          --node-index 4
          --rpc-host 0.0.0.0 --rpc 8004
          --gossip-host 0.0.0.0 --gossip 9004
          --peers node0:9000,node1:9001,node2:9002,node3:9003
          --log /data/node4.log
      "
    ports:
      - "8004:8004"
'''
(ROOT / "docker-compose.yml").write_text(docker_compose, encoding="utf-8")
print("Wrote:", ROOT / "docker-compose.yml")


## 3) Write `Makefile`

In [ ]:

import pathlib

ROOT = pathlib.Path("numchain").resolve()

makefile_txt = r'''.PHONY: genesis up down logs flood poll metrics clean

GENESIS=genesis.json

genesis:
	python numchain_v142.py deps
	python numchain_v142.py genesis --out $(GENESIS) --nodes 5 --chain-id numchain-dev --shards 8 --bond 1000

up:
	docker compose up --build

down:
	docker compose down --remove-orphans

logs:
	docker compose logs -f

flood:
	python numchain_v142.py flood --url http://127.0.0.1:8000 --genesis $(GENESIS) --seconds 60 --target-tps 4000 --senders 512 --ramp-up 10

poll:
	python numchain_v142.py poll --urls http://127.0.0.1:8000,http://127.0.0.1:8001,http://127.0.0.1:8002,http://127.0.0.1:8003,http://127.0.0.1:8004 --seconds 60 --every 2

metrics:
	@echo "Open:"
	@echo "  http://127.0.0.1:8000/health"
	@echo "  http://127.0.0.1:8000/metrics"
	@echo "  http://127.0.0.1:8001/metrics"
	@echo "  http://127.0.0.1:8002/metrics"
	@echo "  http://127.0.0.1:8003/metrics"
	@echo "  http://127.0.0.1:8004/metrics"

clean:
	rm -rf logs
	rm -f *.log
'''
(ROOT / "Makefile").write_text(makefile_txt, encoding="utf-8")
print("Wrote:", ROOT / "Makefile")


## 4) Write `README.md`

In [ ]:

import pathlib

ROOT = pathlib.Path("numchain").resolve()

readme_txt = '''# NUMCHAIN v1.4.2 (Closed-Alpha Kernel)

## What this is
A minimal multi-node chain kernel intended for closed-alpha testing:
- Deterministic genesis (reproducible devnet)
- Ed25519 tx signatures
- Per-shard MA (Memory Arithmetic) state transition engine
- Minimal BFT-like flow (proposal → votes → QC → commit)
- TCP gossip over localhost/LAN
- Reliable delivery (ACK + bounded retry) for consensus-critical messages
- Tx propagation: submit to one node → gossiped to peers (TTL/hops)
- Append-only JSONL log for restart + replay
- Minimal RPC surface + Prometheus /metrics

## What this is not
- Not a WAN-hardened P2P network (no libp2p, NAT traversal, encryption policy)
- Not a full public testnet
- Not a production DB (no RocksDB / LevelDB)
- Not a finalized consensus spec (this is a kernel for experimentation)

## Quickstart (5-node localhost)
Prereqs: Docker + docker compose.

### 1) Generate deterministic genesis
```bash
make genesis
```

### 2) Start the 5-node cluster
```bash
make up
```

RPC endpoints:
- node0 http://127.0.0.1:8000
- node1 http://127.0.0.1:8001
- node2 http://127.0.0.1:8002
- node3 http://127.0.0.1:8003
- node4 http://127.0.0.1:8004

Health/metrics:
- http://127.0.0.1:8000/health
- http://127.0.0.1:8000/metrics

### 3) Flood one node (tx gossip spreads to others)
```bash
make flood
```

### 4) Poll the cluster
```bash
make poll
```

## Minimal RPC
- POST `/tx`
- GET `/health`
- GET `/metrics`
- GET `/block/{height}`
- GET `/receipt/{txid}`

## Known limitations
- Single-process per node; CPU saturation impacts gossip + RPC + block production
- No fee market / tx prioritization; mempool is FIFO-ish
- Best-effort gossip transport (ACK+retry only for critical consensus messages)
- No validator onboarding or set changes
- No light-client sync model; nodes trust genesis + peers + log replay
- Long runs may accumulate memory in Python dicts (prototype hygiene)
'''
(ROOT / "README.md").write_text(readme_txt, encoding="utf-8")
print("Wrote:", ROOT / "README.md")


## 5) Write helper scripts under `tools/`

In [ ]:

import pathlib, os

ROOT = pathlib.Path("numchain").resolve()
TOOLS = ROOT / "tools"
TOOLS.mkdir(parents=True, exist_ok=True)

flood_sh = '''#!/usr/bin/env bash
set -euo pipefail
python numchain_v142.py flood \
  --url http://127.0.0.1:8000 \
  --genesis genesis.json \
  --seconds 60 \
  --target-tps 4000 \
  --senders 512 \
  --ramp-up 10
'''

poll_sh = '''#!/usr/bin/env bash
set -euo pipefail
python numchain_v142.py poll \
  --urls http://127.0.0.1:8000,http://127.0.0.1:8001,http://127.0.0.1:8002,http://127.0.0.1:8003,http://127.0.0.1:8004 \
  --seconds 60 \
  --every 2
'''

(TOOLS / "flood.sh").write_text(flood_sh, encoding="utf-8")
(TOOLS / "poll.sh").write_text(poll_sh, encoding="utf-8")

os.chmod(TOOLS / "flood.sh", 0o755)
os.chmod(TOOLS / "poll.sh", 0o755)

print("Wrote:", TOOLS / "flood.sh")
print("Wrote:", TOOLS / "poll.sh")


## 6) Sanity-check scaffold

In [ ]:

import pathlib

ROOT = pathlib.Path("numchain").resolve()
required = [
    "numchain_v142.py",
    "docker-compose.yml",
    "Makefile",
    "README.md",
    "tools/flood.sh",
    "tools/poll.sh",
]
missing = [r for r in required if not (ROOT / r).exists()]
if missing:
    raise SystemExit("Missing: " + ", ".join(missing))
print("All scaffold files present.")
print("Repo ready at:", ROOT)


## 7) Run (manual)
Run these in a terminal **from inside `./numchain/`**:

```bash
cd numchain
make genesis
make up
# in another terminal
cd numchain
make flood
# in another terminal
cd numchain
make poll
```
